##Imports

In [0]:
from pyspark.sql.functions import col

##Puxando todas as tabelas dimensão

In [0]:
df_silver = spark.sql("SELECT * FROM controlled_medication_data.silver_layer.vendas")
df_dim_data = spark.sql("SELECT * FROM controlled_medication_data.gold_layer.dim_data")
df_dim_localizacao = spark.sql("SELECT * FROM controlled_medication_data.gold_layer.dim_localizacao_venda")
df_dim_produto = spark.sql("SELECT * FROM controlled_medication_data.gold_layer.dim_produto")
df_dim_prescritor = spark.sql("SELECT * FROM controlled_medication_data.gold_layer.dim_prescritor")
df_dim_comprador = spark.sql("SELECT * FROM controlled_medication_data.gold_layer.dim_comprador")

In [0]:
df_fat_vendas = df_silver

## Realizando União (left-join) de cada dim na tabela silver

In [0]:
df_fat_vendas = df_fat_vendas.join(
    df_dim_data,
    on="data_venda",
    how="left"
).select(
    df_fat_vendas["*"],
    col("id_data")
)

In [0]:
df_fat_vendas.select("data_venda", "id_data").distinct().display()

In [0]:
df_fat_vendas = df_fat_vendas.join(
    df_dim_localizacao,
    on=["uf_venda", "municipio_venda", "latitude_municipio", "longitude_municipio", "capital", "nome_uf", "latitude_uf", "longitude_uf", "regiao"],
    how="left"
).select(
    df_fat_vendas["*"],
    col("id_localizacao")
)

In [0]:
df_fat_vendas = df_fat_vendas.join(
    df_dim_produto,
    on=["principio_ativo", "descricao_apresentacao", "unidade_medida", "CID10"],
    how="left"
).select(
    df_fat_vendas["*"],
    col("id_produto")
)

In [0]:
df_fat_vendas = df_fat_vendas.join(
    df_dim_prescritor,
    on=["conselho_prescritor", "uf_conselho_prescritor", "tipo_receituario"],
    how="left"
).select(
    df_fat_vendas["*"],
    col("id_prescritor")
)

In [0]:
df_fat_vendas = df_fat_vendas.join(
    df_dim_comprador,
    on=["sexo_comprador", "idade_comprador"],
    how="left"
).select(
    df_fat_vendas["*"],
    col("id_comprador")
)

## Criação da tabela fato, apenas com ids das fatos e fato da venda (quantidade_vendida)

In [0]:
df_fat_vendas_formatado = df_fat_vendas.select(
    "qtd_vendida",
    "id_data",
    "id_localizacao",
    "id_produto",
    "id_prescritor",
    "id_comprador"
)

## Tratamento de ids nulos

In [0]:
campos_com_nulos = ["id_comprador", "id_data", "id_localizacao", "id_produto", "id_prescritor"]

In [0]:
df_fat_vendas = df_fat_vendas_formatado.fillna(-1, campos_com_nulos)

In [0]:
df_fat_vendas.count()

## Salvando tabela

In [0]:
df_fat_vendas_final.write.mode("overwrite").saveAsTable("controlled_medication_data.gold_layer.ft_vendas")